In [ ]:
## Add new column ##
df_renamed = df_all.withColumnRenamed("RatecodeID", "Rate_codeID") 
df_renamed.show(5)

In [ ]:
df_dropped = df_all.drop("passenger_count_plus_10")
df_dropped.show(5) 

In [ ]:
df_all.groupBy("VendorID").agg({"passenger_count": "avg", "extra": "max"}).show()

In [ ]:
## Sampling Data
SAMPLE_SIZE = 0.01
df = df_all.sample(SAMPLE_SIZE, seed = 20020223).toPandas() 
df

In [ ]:
df_2019_1.printSchema() 
print(f"数据总量: {df_2019_1.count()}") 
df_2019_1.describe().show()

In [ ]:
## missing data 
df_2019_1 = df_2019_1.dropna()
df_2019_1 = df_2019_1.fillna({'passenger_count': 1, 'trip_distance': 0.0})

In [ ]:
# datatype conversion
df_2019_1 = df_2019_1.withColumn("passenger_count", col("passenger_count").cast("integer"))
df_2019_1 = df_2019_1.withColumn("tpep_pickup_datetime", col("tpep_pickup_datetime")) 
df_2019_1 = df_2019_1.withColumn("tpep_dropoff_datetime", col("tpep_dropoff_datetime"))  

In [ ]:
# duplicate data
df_2019_1 = df_2019_1.dropDuplicates()

In [ ]:
# anomaly data
trip_distance_min = 0
trip_distance_max = 100
df_2019_1 = df_2019_1.filter((df_2019_1["trip_distance"] > trip_distance_min) & (df_2019_1["trip_distance"] > trip_distance_max))

In [ ]:
# standardization
from pyspark.ml.feature import StandardScaler, VectorAssembler
assembler = VectorAssembler(inputCols=["trip_distance", "trip_duration"], outputCol= "features")
df_features = assembler.transform(df_2019_1)
scaler = StandardScaler(inputCol="features", outputCol="scaled_features") 
scaler_model = scaler.fit(df_features)
df_scaled = scaler_model.transform(df_features)